In [19]:
import requests
import scrapy
import re
import json
REQ_HEADERS={'user-agent': 'UNC MEJO 583'}
resp = requests.get("http://goheels.com/roster.aspx?path=mbball", headers=REQ_HEADERS)
sel = scrapy.Selector(text=resp.text)

In [20]:
def loadPlayer(player_id):
    res = requests.get("http://goheels.com/roster.aspx?rp_id="+player_id, headers=REQ_HEADERS)
    page = scrapy.Selector(text=res.text)
    info = page.css('.sidearm-roster-player-fields')
    headers = info.css('dt').xpath('string()').extract()
    data = info.css('dd').xpath('string()').extract()
    bio = dict(zip(headers, data))
    names = page.css('.sidearm-roster-player-name span::text').extract()
    bio['First Name'] = names[0]
    bio['Last Name'] = names[1]
    bio['Full Name'] = ' '.join(names)
    bio['Number'] = page.css('.sidearm-roster-player-jersey-number::text').extract_first()
    bio['Image'] = page.css('.sidearm-roster-player-image img::attr(src)').extract_first()
    return bio


In [21]:
def loadStats(player_id):
    stats_response = requests.get("http://goheels.com/services/responsive-roster-bio.ashx?type=stats&path=mbball&year=2017&rp_id="+player_id, headers=REQ_HEADERS).json()
    current_stats = scrapy.Selector(text=stats_response['current_stats'])
    stats_headers = current_stats.css('th').xpath('string()').extract()
    stats_numbers = current_stats.css('tfoot td').xpath('string()').extract()
    stats = dict(zip(stats_headers, stats_numbers))
    if 'Opponent' in stats: del stats['Opponent']
    if 'GS' in stats: del stats['GS']
    if 'Date' in stats: del stats['Date']
    return stats

In [22]:
table = sel.css('table')[0]
headers = table.css('th').xpath('string()').extract()
players = []
for href in table.xpath('//tr//td//a/@href').extract():
    if 'roster.aspx' in href:
        player_id = re.search('rp_id=(\d+)', href).group(1)
        bio = loadPlayer(player_id)
        bio['Stats'] = loadStats(player_id)
        players.append(bio)

In [23]:
players

[{'Class': 'Sophomore',
  'First Name': 'Seventh',
  'Full Name': 'Seventh Woods',
  'Height': '6-1',
  'High School': 'Hammond School',
  'Hometown': 'Columbia, S.C.',
  'Image': '/images/2017/9/12/Woods_Seventh_mb_2017_181.jpg?width=300',
  'Last Name': 'Woods',
  'Number': '0',
  'Position': 'Guard',
  'Stats': {'%': '.750',
   '3FG/A': '0-2',
   'AST': '12',
   'AVG': '2.4',
   'BLK': '0',
   'DEF': '10',
   'FGM/A': '7-20',
   'FTM/A': '3-4',
   'MIN': '76',
   'OFF': '0',
   'PF': '5',
   'PTS': '17',
   'STL': '4',
   'T/O': '5',
   'TOT': '10'},
  'Weight': '185'},
 {'Class': 'Senior',
  'First Name': 'Theo',
  'Full Name': 'Theo Pinson',
  'Height': '6-6',
  'High School': 'Wesleyan Christian Academy',
  'Hometown': 'Greensboro, N.C.',
  'Image': '/images/2017/9/12/Pinson_Theo_mb_2017_23.jpg?width=300',
  'Last Name': 'Pinson',
  'Number': '1',
  'Position': 'Forward/Guard',
  'Stats': {'%': '.810',
   '3FG/A': '5-34',
   'AST': '105',
   'AVG': '8.8',
   'BLK': '14',
   'DEF'

In [24]:
with open('heels.json', 'w') as jsonfile:
    json.dump(players, jsonfile)